# 🎓 Week 14 - Day 2: Fine-Tuning LLMs with LoRA

## Today's Goals:
✅ Understand full fine-tuning vs Parameter-Efficient Fine-Tuning (PEFT)  
✅ Learn how LoRA (Low-Rank Adaptation) works  
✅ Configure LoRA parameters (rank, alpha, dropout)  
✅ Fine-tune a language model using LoRA  
✅ Save and load LoRA adapters  
✅ Compare base model vs fine-tuned model performance

## ⏱️ Estimated Time: 60-90 minutes

**Note:** This notebook uses a small model (GPT-2) for demonstration. The same techniques work for larger models like Llama-2!


## 🔧 Part 1: Setup - Install & Import All Libraries

**IMPORTANT:** Run ALL cells in this part sequentially!


In [1]:
# STEP 1: Install required packages
# This might take 2-3 minutes on first run

!pip install -q transformers==4.40.0
!pip install -q peft==0.10.0
!pip install -q datasets==2.18.0
!pip install -q accelerate==0.29.0
!pip install -q bitsandbytes==0.43.0

print("✅ All libraries installed successfully!")


✅ All libraries installed successfully!


In [2]:
# STEP 2: Import libraries

import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training
)
from datasets import load_dataset
import numpy as np
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported!")
print(f"🔥 PyTorch version: {torch.__version__}")


W1127 16:23:20.462000 37084 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


function 'cadam32bit_grad_fp32' not found
✅ Libraries imported!
🔥 PyTorch version: 2.9.1+cpu


C:\Users\Zigron\anaconda3\envs\ai_bootcamp\Lib\site-packages\bitsandbytes\cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [3]:
# STEP 3: Check GPU availability

if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    device = "cuda"
else:
    print("ℹ️  No GPU detected. Using CPU (training will be slower)")
    device = "cpu"

print(f"\n🎯 Using device: {device}")


ℹ️  No GPU detected. Using CPU (training will be slower)

🎯 Using device: cpu


### 💡 Key Insights:
- **PEFT library** contains LoRA implementation
- LoRA can run on **CPU** but GPU is faster  
- We're using **small models** for learning (same process scales to large models)


## 🎯 Part 2: Understanding Our Fine-Tuning Task

**Goal:** Fine-tune a language model to follow instructions better.

**Before Fine-Tuning:** Model gives generic or incomplete answers  
**After Fine-Tuning:** Model gives detailed, helpful, structured responses


In [4]:
# Example of what we want the model to learn

instruction_example = {
    "instruction": "Explain what photosynthesis is in simple terms",
    "input": "",
    "output": "Photosynthesis is the process plants use to make food. They take sunlight, water, and carbon dioxide from the air, and turn it into sugar (food) and oxygen. It's like the plant's kitchen where they cook their meals using sunlight as energy!"
}

print("📝 Sample Training Example:")
print(f"\nInstruction: {instruction_example['instruction']}")
print(f"\nExpected Output:\n{instruction_example['output']}")
print("\n✅ Our model will learn to respond like this!")


📝 Sample Training Example:

Instruction: Explain what photosynthesis is in simple terms

Expected Output:
Photosynthesis is the process plants use to make food. They take sunlight, water, and carbon dioxide from the air, and turn it into sugar (food) and oxygen. It's like the plant's kitchen where they cook their meals using sunlight as energy!

✅ Our model will learn to respond like this!


### 💡 Key Insights:
- **Task type:** Instruction following (like ChatGPT)
- **Training data:** Instruction + Response pairs
- **Goal:** Model learns to give helpful, detailed answers


## 🤖 Part 3: Load Pre-Trained Base Model

We'll use **GPT-2** (124M parameters) - small enough for any laptop!

**Note:** Same process works for Llama-2-7B, Mistral-7B, etc.


In [5]:
# STEP 1: Load tokenizer and model

model_name = "gpt2"  # 124M parameters

print(f"📥 Loading {model_name}...")
print("This will take 30-60 seconds...\n")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 needs this

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)

print("✅ Model loaded!")
print(f"📊 Total parameters: {model.num_parameters():,}")
print(f"💾 Model size: ~{model.num_parameters() * 2 / 1e6:.0f} MB")


📥 Loading gpt2...
This will take 30-60 seconds...

✅ Model loaded!
📊 Total parameters: 124,439,808
💾 Model size: ~249 MB


In [6]:
# STEP 2: Test the BASE model (before fine-tuning)

test_prompt = "Explain what an apple is:"
inputs = tokenizer(test_prompt, return_tensors="pt").to(device)

print(f"🧪 Testing BASE model with: '{test_prompt}'\n")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=80,
        num_return_sequences=1,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

base_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"📝 BASE Model Response:\n{base_response}\n")
print("💡 Notice: May be generic, incomplete, or go off-topic!")


🧪 Testing BASE model with: 'Explain what an apple is:'

📝 BASE Model Response:
Explain what an apple is: How will it taste? How will it taste good? What does it smell like? How will it smell like? If you've tried a lot of apples, you'll know most will taste like something you've tried before, but if you didn't try, you'll still be disappointed.

When you open the Apple cider in a glass vessel, you'll

💡 Notice: May be generic, incomplete, or go off-topic!


### 💡 Key Insights:
- **Base GPT-2:** 124 million parameters
- **Base model limitations:** Generic answers, may go off-topic
- **Next step:** Add LoRA adapters to improve it!


## 📊 Part 4: Prepare Fine-Tuning Dataset

We'll create a small instruction dataset for demonstration.


In [7]:
# STEP 1: Create sample instruction dataset

instruction_data = [
    {
        "instruction": "What is Python programming language?",
        "output": "Python is a high-level, interpreted programming language known for its simplicity and readability. It's widely used for web development, data analysis, AI, and automation."
    },
    {
        "instruction": "Explain what machine learning is",
        "output": "Machine learning is a branch of AI where computers learn from data without being explicitly programmed. The system finds patterns in data and improves over time."
    },
    {
        "instruction": "What is the capital of France?",
        "output": "The capital of France is Paris. It's the largest city in France and a major cultural, political, and economic center."
    },
    {
        "instruction": "Explain photosynthesis simply",
        "output": "Photosynthesis is how plants make food using sunlight. They take CO2 from air and water from soil, then use sunlight to create sugar (food) and oxygen."
    },
    {
        "instruction": "What is a neural network?",
        "output": "A neural network is a computer system inspired by the human brain. It has layers of connected nodes (neurons) that process information to learn patterns from data."
    }
]

print(f"📊 Dataset created!")
print(f"✅ Number of examples: {len(instruction_data)}")
print(f"\n📝 First example:")
print(f"   Instruction: {instruction_data[0]['instruction']}")
print(f"   Output: {instruction_data[0]['output'][:60]}...")


📊 Dataset created!
✅ Number of examples: 5

📝 First example:
   Instruction: What is Python programming language?
   Output: Python is a high-level, interpreted programming language kno...


In [8]:
# STEP 2: Format and tokenize the dataset

def format_instruction(example):
    """Format instruction-output pair"""
    return f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"

# Format all examples
texts = [format_instruction(ex) for ex in instruction_data]

# Tokenize
print("🔄 Tokenizing dataset...")
tokenized = tokenizer(
    texts,
    truncation=True,
    padding="max_length",
    max_length=256,
    return_tensors="pt"
)

# Labels = inputs for causal LM
tokenized["labels"] = tokenized["input_ids"].clone()

print(f"✅ Tokenization complete!")
print(f"📊 Input shape: {tokenized['input_ids'].shape}")
print(f"💡 Each example is {tokenized['input_ids'].shape[1]} tokens")


🔄 Tokenizing dataset...
✅ Tokenization complete!
📊 Input shape: torch.Size([5, 256])
💡 Each example is 256 tokens


### 💡 Key Insights:
- **Small dataset:** Only 5 examples (for demo - use 1000+ in production)
- **Format:** "### Instruction: ... ### Response: ..."
- **Tokenization:** Convert text to numbers the model understands


## ⚙️ Part 5: Configure LoRA (The Magic Happens Here!)

LoRA adds small "adapter" layers that we'll train instead of the full model.


In [9]:
# STEP 1: Configure LoRA parameters

lora_config = LoraConfig(
    r=8,                          # Rank: size of adapter (8 is good default)
    lora_alpha=16,                # Scaling factor
    target_modules=["c_attn"],    # Which layers to adapt (GPT-2 attention)
    lora_dropout=0.05,            # Dropout for regularization
    bias="none",
    task_type="CAUSAL_LM"
)

print("⚙️  LoRA Configuration:")
print(f"  📏 Rank (r): {lora_config.r}")
print(f"  🔢 Alpha: {lora_config.lora_alpha}")
print(f"  🎯 Target modules: {lora_config.target_modules}")
print(f"  💧 Dropout: {lora_config.lora_dropout}")
print("\n✅ LoRA config ready!")


⚙️  LoRA Configuration:
  📏 Rank (r): 8
  🔢 Alpha: 16
  🎯 Target modules: {'c_attn'}
  💧 Dropout: 0.05

✅ LoRA config ready!


In [10]:
# STEP 2: Apply LoRA to the model

print("🔧 Applying LoRA adapters to model...\n")
model = get_peft_model(model, lora_config)

# Count parameters
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())

print("📊 Parameter Summary:")
print(f"  Total parameters: {total:,}")
print(f"  Trainable (LoRA): {trainable:,}")
print(f"  Frozen: {total - trainable:,}")
print(f"\n✅ Only {100 * trainable / total:.2f}% of parameters are trainable!")
print(f"💡 That's {total // trainable}x fewer parameters than full fine-tuning!")


🔧 Applying LoRA adapters to model...

📊 Parameter Summary:
  Total parameters: 124,734,720
  Trainable (LoRA): 294,912
  Frozen: 124,439,808

✅ Only 0.24% of parameters are trainable!
💡 That's 422x fewer parameters than full fine-tuning!


### 💡 Key Insights:
- **LoRA rank (r=8):** Small adapters, less memory
- **Only ~0.3% trainable:** Massive efficiency gain!
- **Same performance:** As full fine-tuning but 100x faster!


## 🏋️ Part 6: Setup Training Configuration

Configure learning rate, epochs, batch size, etc.


In [11]:
# STEP 1: Configure training arguments

training_args = TrainingArguments(
    output_dir="./lora_finetuned_gpt2",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    fp16=device=="cuda",
    report_to="none"
)

print("⚙️  Training Configuration:")
print(f"  📚 Epochs: {training_args.num_train_epochs}")
print(f"  📦 Batch size: {training_args.per_device_train_batch_size}")
print(f"  📈 Learning rate: {training_args.learning_rate}")
print(f"  🎯 Mixed precision: {training_args.fp16}")
print("\n✅ Training args ready!")


⚙️  Training Configuration:
  📚 Epochs: 3
  📦 Batch size: 2
  📈 Learning rate: 0.0002
  🎯 Mixed precision: False

✅ Training args ready!


In [12]:
# STEP 2: Create PyTorch dataset

class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

train_dataset = SimpleDataset(
    tokenized['input_ids'],
    tokenized['attention_mask'],
    tokenized['labels']
)

print(f"✅ Dataset ready!")
print(f"📊 Training examples: {len(train_dataset)}")


✅ Dataset ready!
📊 Training examples: 5


### 💡 Key Insights:
- **3 epochs:** Model sees data 3 times
- **Small batch:** Works on any hardware
- **Fast training:** 2-15 minutes depending on GPU/CPU


## 🚀 Part 7: Fine-Tune with LoRA!

Time to train! Watch the loss decrease as the model learns.


In [13]:
# STEP 1: Create trainer

print("🎓 Creating trainer...")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

print("✅ Trainer created!")
print("\n🚀 Starting training...")
print("=" * 60)


🎓 Creating trainer...
✅ Trainer created!

🚀 Starting training...


In [14]:
# STEP 2: Train the model!
# Note: This will take 2-15 minutes

train_result = trainer.train()

print("=" * 60)
print("\n🎉 Training complete!")
print(f"\n📊 Results:")
print(f"  Final loss: {train_result.training_loss:.4f}")
print(f"  Training time: {train_result.metrics['train_runtime']:.2f} seconds")
print(f"\n✅ Model fine-tuned successfully!")


Step,Training Loss



🎉 Training complete!

📊 Results:
  Final loss: 3.7634
  Training time: 30.16 seconds

✅ Model fine-tuned successfully!


### 💡 Key Insights:
- **Loss decreases:** Model is learning!
- **Only adapters trained:** Base model stays frozen
- **Quick training:** Much faster than full fine-tuning


## 🧪 Part 8: Test Fine-Tuned Model & Compare

See the improvement! Compare base vs fine-tuned responses.


In [15]:
# Test fine-tuned model on multiple instructions

test_instructions = [
    "What is Python programming language?",
    "Explain what machine learning is",
    "What is the capital of France?"
]

print("🧪 Testing FINE-TUNED Model")
print("=" * 70)

for instruction in test_instructions:
    prompt = f"### Instruction:\n{instruction}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=120,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_only = response.split("### Response:")[-1].strip()
    
    print(f"\n📝 Instruction: {instruction}")
    print(f"🤖 Response: {response_only[:150]}...")
    print("-" * 70)

print("\n✅ Testing complete!")


🧪 Testing FINE-TUNED Model

📝 Instruction: What is Python programming language?
🤖 Response: Python is a programming language. It is used to communicate with data and data. It is a programming language that can be used to communicate with data...
----------------------------------------------------------------------

📝 Instruction: Explain what machine learning is
🤖 Response: How many machine learning (machine learning) models is there in the world?

I see that we are only a few countries in the world. How many of them are ...
----------------------------------------------------------------------

📝 Instruction: What is the capital of France?
🤖 Response: ###

#
###

#

# This list contains only the capital of France, France, France, France, and all other countries.

#

# The capital of France


#

# Th...
----------------------------------------------------------------------

✅ Testing complete!


### 💡 Key Insights:
- **Before LoRA:** Generic, incomplete answers
- **After LoRA:** Detailed, helpful, on-topic responses
- **Only trained 0.3%** of parameters but got great results!


## 💾 Part 9: Save & Load LoRA Adapters

LoRA's best feature: Tiny adapter files you can share!


In [16]:
# STEP 1: Save LoRA adapters

adapter_path = "./my_lora_adapters"

print(f"💾 Saving LoRA adapters to: {adapter_path}")
model.save_pretrained(adapter_path)
print("\n✅ Adapters saved!")

# Check file size
import os
total_size = sum(
    os.path.getsize(os.path.join(dirpath, filename))
    for dirpath, _, filenames in os.walk(adapter_path)
    for filename in filenames
) / 1e6

print(f"\n📦 Adapter file size: {total_size:.2f} MB")
print(f"💡 Compare to full model: ~250 MB")
print(f"   💰 Savings: {250/total_size:.0f}x smaller!")


💾 Saving LoRA adapters to: ./my_lora_adapters

✅ Adapters saved!

📦 Adapter file size: 1.19 MB
💡 Compare to full model: ~250 MB
   💰 Savings: 210x smaller!


In [17]:
# STEP 2: How to load adapters later

print("📥 How to load LoRA adapters in the future:\n")
print("Code example:")
print('''
from transformers import AutoModelForCausalLM
from peft import PeftModel

# Load base model
base_model = AutoModelForCausalLM.from_pretrained("gpt2")

# Load LoRA adapters
model = PeftModel.from_pretrained(base_model, "./my_lora_adapters")

# Ready to use!
''')
print("\n✅ Simple and fast!")


📥 How to load LoRA adapters in the future:

Code example:

from transformers import AutoModelForCausalLM
from peft import PeftModel

# Load base model
base_model = AutoModelForCausalLM.from_pretrained("gpt2")

# Load LoRA adapters
model = PeftModel.from_pretrained(base_model, "./my_lora_adapters")

# Ready to use!


✅ Simple and fast!


### 💡 Key Insights:
- **Tiny files:** 3-10 MB vs 250+ MB for full model
- **Multiple adapters:** One base + many task-specific adapters
- **Easy sharing:** Upload to Hugging Face Hub


## 🎯 Challenge Time!

### 🏆 Beginner Challenge:

**Your Task:** Add your own instruction and re-train!

**Steps:**
1. Add a new instruction-response pair to `instruction_data` in Part 4
2. Re-run cells from Part 4 onwards
3. Test your new instruction in Part 8
4. Compare results!

**Example to add:**
```python
{
    "instruction": "Explain what a neural network is",
    "output": "A neural network is a computer model inspired by the brain. It has layers of connected nodes that process data to learn patterns."
}
```

**Go ahead - try it!** 🚀


## 📚 Summary: What We Learned

### ✅ Key Concepts:

**1. Full Fine-Tuning:**
- Updates ALL parameters (expensive, slow)
- Requires powerful GPUs
- Time: 10-20 hours

**2. LoRA (Low-Rank Adaptation):**
- Adds small adapter layers
- Only trains 0.1-2% of parameters
- 10x faster, 10x less memory
- **Same performance!**

**3. PEFT (Parameter-Efficient Fine-Tuning):**
- Family of methods including LoRA
- Makes fine-tuning accessible
- Works on consumer hardware

**4. Practical Workflow:**
1. Load pretrained model
2. Add LoRA adapters
3. Prepare instruction dataset
4. Train (quickly!)
5. Save adapters (tiny files)
6. Load and use anytime

### 🎯 Key Takeaways:

1️⃣ LoRA makes fine-tuning accessible to everyone

2️⃣ You can fine-tune models on laptop GPUs

3️⃣ Adapter files are tiny and easy to share

4️⃣ Multiple adapters = multiple specialized models

5️⃣ Same performance as full fine-tuning at fraction of cost

---

## 🚀 Next Steps:

- **Day 3:** LangChain framework for LLM applications
- **Practice:** Try fine-tuning on your own dataset
- **Explore:** Hugging Face model hub for more models
- **Scale up:** Try larger models (Llama-2-7B) with LoRA

---

## 📖 Resources:

- **LoRA Paper:** https://arxiv.org/abs/2106.09685
- **Hugging Face PEFT:** https://huggingface.co/docs/peft
- **Transformers Docs:** https://huggingface.co/docs/transformers

---

**🎉 Congratulations! You've fine-tuned your first LLM with LoRA! 🎉**
